In [79]:
#!pip install tensorflow

In [80]:
#!pip install tensorboard

In [81]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime

import tensorboard

import tqdm


In [82]:
# 1. get mnist from tensorflow_datasets
mnist = tfds.load("mnist", split =["train","test"], as_supervised=True)
train_ds = mnist[0]
val_ds = mnist[1]

In [83]:
# 2. write function to create the dataset that we want
def preprocess_task1(data, batch_size):
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, float), t))
    # image should be flattened
    data = data.map(lambda x, t: (tf.reshape(x, (-1,)), t))
    # image vector will here have values between -1 and 1
    data = data.map(lambda x,t: ((x/128.)-1., t))
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), 
                                     data.shuffle(2000)))
    # map ((x1,y1),(x2,y2)) to (x1,x2, y1==y2*) *boolean
    zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1] + x2[1] >= 5))
    # transform boolean target to int
    zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.int32)))
    # batch the dataset
    zipped_ds = zipped_ds.batch(batch_size)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds


In [84]:
# 2. write function to create the dataset that we want
def preprocess_task2(data, batch_size):
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, float), t))
    # image should be flattened
    data = data.map(lambda x, t: (tf.reshape(x, (-1,)), t))
    # image vector will here have values between -1 and 1
    data = data.map(lambda x,t: ((x/128.)-1., t))
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), 
                                     data.shuffle(2000)))
    # map ((x1,y1),(x2,y2)) to (x1,x2, y1==y2*) *boolean
    zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1] - x2[1]))
   
    zipped_ds = zipped_ds.map(lambda img1, img2, target: (img1, img2, tf.one_hot(target, depth=38)))
    # batch the dataset
    zipped_ds = zipped_ds.batch(batch_size)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds

In [85]:
#train_ds = preprocess_task1(train_ds, batch_size=32) #train_ds.apply(preprocess)
#val_ds = preprocess_task1(val_ds, batch_size=32) #val_ds.apply(preprocess)

In [86]:
'''# check the contents of the dataset
for img1, img2, label in train_ds.take(1):
    print(img1.shape, img2.shape, label.shape)

train_ds.map(lambda x1, x2, t: (x1, x2, tf.cast(t, tf.int32)))'''

'# check the contents of the dataset\nfor img1, img2, label in train_ds.take(1):\n    print(img1.shape, img2.shape, label.shape)\n\ntrain_ds.map(lambda x1, x2, t: (x1, x2, tf.cast(t, tf.int32)))'

In [87]:
def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    val_log_path = f"logs/{config_name}/{current_time}/val"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for validation metrics
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer

train_summary_writer, val_summary_writer = create_summary_writers(config_name="RUN1")

In [88]:
class TwinMNISTModel(tf.keras.Model):

    # 1. constructor
    def __init__(self, optimizer = "Adam", n_of_task = 1 ):
        super().__init__()
        # inherit functionality from parent class

        # optimizer, loss function and metrics
        self.metrics_list = [tf.keras.metrics.BinaryAccuracy(),
                             tf.keras.metrics.Mean(name="loss")] 
        
        if(optimizer == "Adam"):
          self.optimizer = tf.keras.optimizers.Adam()
        elif(optimizer == "SGD"):
          self.optimizer = tf.keras.optimizers.SGD()

        if n_of_task == 1:
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
        if n_of_task == 2:
            self.loss_function = tf.keras.losses.CategoricalCrossentropy()

        # layers to encode the images (both layers used for both images)
        self.dense1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        
        self.dense3 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        
        self.out_layer = tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)
        
    # 2. call method (forward computation)
    def call(self, images, training=False):
        img1, img2 = images
        
        img1_x = self.dense1(img1)
        img1_x = self.dense2(img1_x)
        
        img2_x = self.dense1(img2)
        img2_x = self.dense2(img2_x)
        
        combined_x = tf.concat([img1_x, img2_x ], axis=1)
        combined_x = self.dense3(combined_x)
        return self.out_layer(combined_x)

    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model

    # 4. reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    # 5. train step method
    @tf.function
    def train_step(self, data):
        img1, img2, label = data
        
        with tf.GradientTape() as tape:
            output = self((img1, img2), training=True)
            loss = self.loss_function(label, output)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update the state of the metrics according to loss
        self.metrics[0].update_state(label, output)
        self.metrics[1].update_state(loss)
        
        # return a dictionary with metric names as keys and metric results as values
        return {m.name : m.result() for m in self.metrics}

    # 6. test_step method
    @tf.function
    def test_step(self, data):
        img1, img2, label = data
        # same as train step (without parameter updates)
        output = self((img1, img2), training=False)
        loss = self.loss_function(label, output)
        self.metrics[0].update_state(label, output)
        self.metrics[1].update_state(loss)
        
        return {m.name : m.result() for m in self.metrics}

In [89]:
def training_loop(model, train_ds, val_ds, start_epoch, epochs, train_summary_writer, val_summary_writer, save_path):
    for e in range(start_epoch, epochs):
        for data in tqdm.tqdm(train_ds, position=0, leave=True):
            metrics = model.train_step(data)

        # 3. log and print training metrics

        with train_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=e)
            # alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)
            # e.g. tf.summary.image(name="mean_activation_layer3", data = metrics["mean_activation_layer3"],step=e)
        
        #print the metrics
        print([f"{key}: {value.numpy()}" for (key, value) in metrics.items()]) 

        # 4. reset metric objects
        model.reset_metrics()


        # 5. evaluate on validation data
        for data in val_ds:
            metrics = model.test_step(data)


        # 6. log validation metrics

        with val_summary_writer.as_default():
            # for scalar metrics:
            for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=e)
            # alternatively, log metrics individually (allows for non-scalar metrics such as tf.keras.metrics.MeanTensor)
            # e.g. tf.summary.image(name="mean_activation_layer3", data = metrics["mean_activation_layer3"],step=e)
            
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # 7. reset metric objects
        model.reset_metrics()

    # 8. save model weights if save_path is given
    if save_path:
        model.save_weights(save_path) 

In [ ]:
#training and test data
#Subtask1
train_ds_task1 = preprocess_task1(train_ds, batch_size=32)
val_ds_task1 = preprocess_task1(val_ds, batch_size=32) 

#training and test data
#Subtask2
train_ds_task2 = preprocess_task2(train_ds, batch_size=32)
val_ds_task2 = preprocess_task2(val_ds, batch_size=32) 


# 1. instantiate model
#Adam Optimizer
model1 = TwinMNISTModel("Adam", 1)
model2 = TwinMNISTModel("Adam", 2)

# 2. choose a path to save the weights
save_path = "trained_model_RUN1"

print("Adam optimizer")
print("Subtask1")
training_loop(model=model1,
    train_ds=train_ds_task1,
    val_ds=val_ds_task1,
    start_epoch=0,
    epochs=10,
    train_summary_writer=train_summary_writer,
    val_summary_writer=val_summary_writer,
    save_path=save_path)

print("Subtask2")
training_loop(model=model2,
    train_ds=train_ds_task2,
    val_ds=val_ds_task2,
    start_epoch=0,
    epochs=10,
    train_summary_writer=train_summary_writer,
    val_summary_writer=val_summary_writer,
    save_path=save_path)

print("SGD optimizer")
#SGD Optimizer
model1 = TwinMNISTModel("SGD", 1)
model2 = TwinMNISTModel("SGD", 2)

#Subtask1
training_loop(model=model1,
    train_ds=train_ds_task1,
    val_ds=val_ds_task1,
    start_epoch=0,
    epochs=10,
    train_summary_writer=train_summary_writer,
    val_summary_writer=val_summary_writer,
    save_path=save_path)

print("Subtask2")
#Subtask2
training_loop(model=model2,
    train_ds=train_ds_task2,
    val_ds=val_ds_task2,
    start_epoch=0,
    epochs=10,
    train_summary_writer=train_summary_writer,
    val_summary_writer=val_summary_writer,
    save_path=save_path)


Adam optimizer
Subtask1


100%|██████████| 1875/1875 [00:25<00:00, 73.92it/s] 


['binary_accuracy: 0.9424833059310913', 'loss: 0.15026351809501648']
['val_binary_accuracy: 0.9733999967575073', 'val_loss: 0.07786554098129272']


100%|██████████| 1875/1875 [00:17<00:00, 104.53it/s]


['binary_accuracy: 0.9718833565711975', 'loss: 0.07965642213821411']
['val_binary_accuracy: 0.9757000207901001', 'val_loss: 0.07104188948869705']


...

100%|██████████| 1875/1875 [00:16<00:00, 112.12it/s]


['binary_accuracy: 0.9824000000953674', 'loss: 0.051684293895959854']
['val_binary_accuracy: 0.9815999865531921', 'val_loss: 0.05700656771659851']


100%|██████████| 1875/1875 [00:20<00:00, 91.52it/s] 


['binary_accuracy: 0.9830166697502136', 'loss: 0.04811704158782959']
['val_binary_accuracy: 0.9818999767303467', 'val_loss: 0.05286778509616852']


 42%|████▏     | 788/1875 [00:07<00:09, 112.02it/s]